In [1]:
from pathlib import Path

import automech
from project_utilities import p_, util, workflow
from protomech import mess

file = util.notebook_file() if util.is_notebook() else __file__
tag = util.file_tag(file)
root_path = Path("..")
tag0 = "Z_combined_v0"

In [2]:
mech0 = automech.io.read(p_.stereo_mechanism(tag0, "json", p_.data(root_path)))
mech0, _ = automech.replace_unstable_products(mech0)

/home/avcopan/proj/project-cyclopentane/code/protomech/src/automech/_mech.py:1307: UserWarning: This function ignores stereochemistry.
  warnings.warn("This function ignores stereochemistry.", stacklevel=1)


In [3]:
automech.resonant_unstable_species_names(mech0)

['S(2258)z',
 'S(2258)e',
 'C5O2nexammzzz',
 'C5O2nexammzze',
 'C5O2nexammzez',
 'C5O2nexammzee',
 'C5O2nexammezz',
 'C5O2nexammeze',
 'C5O2nexammeez',
 'C5O2nexammeee',
 'C5O2jrscln']

In [4]:
mess_init_path = p_.mess_initial(root_path)
mess_inp_files = list(mess_init_path.glob("*/mess.inp"))
surfs = [mess.surf.from_mess_input(f) for f in mess_inp_files]
surf = mess.surf.combine(surfs)
# mess.net.display(surf, mech=mech0)

In [5]:
# Set MESS header lines
surf = mess.surf.set_mess_header_temperature_list(
    surf,
    [300, 400, 500, 600, 700, 725, 750, 775, 800, 825, 850, 875, 900, 1000, 1100, 1200],
)
surf = mess.surf.set_mess_header_pressure_list(
    surf, [0.1, 0.32, 1.0, 3.2, 10.0, 32.0, 100.0]
)
surf = mess.surf.set_mess_header_line(
    surf, "WellExtension", "WellExtension      0.0010"
)
surf = mess.surf.set_mess_header_line(
    surf, "ExtensionCorrection", "ExtensionCorrection      0.6"
)

In [6]:
# Turn off fake well extension and unset well extension caps
# (Since the well extension caps are not for the full mechanism, I don't trust them)
surf = mess.surf.set_no_fake_well_extension(surf)
surf = mess.surf.unset_well_extension_cap(surf)

In [7]:
# Drop species that result in unconnected wells
# (These species are not formed in ring-opening reactions and, without
# stereoisomerization rates, do not affect the model.)
drop_labels = ["C5O2pkpfsdzr0", "C5O2sidwaoze", "C5O2sidwaozz", "C5O2pkpfsdzr1"]
drop_keys = mess.surf.node_keys(surf, labels=drop_labels)
print(drop_keys)
surf = mess.surf.remove_nodes(surf, keys=drop_keys)
mech0 = automech.without_species(mech0, spc_names=drop_labels)

[16, 85, 120, 121]


In [8]:
surf = mess.surf.correct_fake_well_energies(surf)

In [9]:
surf = mess.surf.merge_resonant_instabilities(surf, mech0)
mech = automech.merge_resonant_instabilities(mech0, remove=True)
# mess.net.display(surf, mech=mech0)

In [10]:
# Write
print("\nWriting mechanism...")
mech_path = p_.stereo_mechanism(tag, ext="json", path=p_.data(root_path))
print(mech_path)
automech.io.write(mech, mech_path)


Writing mechanism...
../data/Z_mess_v0_ste.json


In [11]:
workflow.prepare_comparison(tag=tag, root_path=root_path)


Reading parent mechanism...

Reading stereo mechanism...
../data/Z_mess_v0_ste.json

Writing mechanism...
../data/Z_mess_v0_comp.json


In [12]:
# Print reagents that are not on the MESS surface
for rgt_names in automech.reaction.reagents(mech.reactions):
    node = next((n for n in surf.nodes if n.names_list == rgt_names), None)
    if node is None:
        print(rgt_names)

In [13]:
surf_dct = mess.surf.split_stoichiometries(surf, mech)
for stoich, stoich_surf in surf_dct.items():
    print(stoich, len(stoich_surf.nodes), len(stoich_surf.edges))

C5H9O2 32 45
C5H9O 18 23
C5H9 6 5
C5H7 9 9
C5H7O2 28 46
C5H11O 4 3
C5H11O2 4 3


In [14]:
mess_final_path = p_.mess_final(root_path)
mess_final_path.mkdir(exist_ok=True)

for stoich, stoich_surf in surf_dct.items():
    print(stoich, len(stoich_surf.nodes), len(stoich_surf.edges))
    stoich_path = mess_final_path / stoich
    stoich_path.mkdir(exist_ok=True)
    mess_inp_file = stoich_path / "mess.inp"
    print(mess_inp_file)
    mess_inp_file.write_text(mess.surf.mess_input(stoich_surf))

    # Test that it can be read back in
    mess.surf.from_mess_input(mess_inp_file)

C5H9O2 32 45
../data/mess/final/C5H9O2/mess.inp
C5H9O 18 23
../data/mess/final/C5H9O/mess.inp
C5H9 6 5
../data/mess/final/C5H9/mess.inp
C5H7 9 9
../data/mess/final/C5H7/mess.inp
C5H7O2 28 46
../data/mess/final/C5H7O2/mess.inp
C5H11O 4 3
../data/mess/final/C5H11O/mess.inp
C5H11O2 4 3
../data/mess/final/C5H11O2/mess.inp


In [15]:
stoich = "C5H7"
stoich_surf = surf_dct[stoich]

mess.net.display(stoich_surf, mech=mech, height="1000px")

In [16]:
print(surf.mess_header)

!===================================================
!  GLOBAL KEYWORDS
!===================================================
TemperatureList[K]                     300.0  400.0  500.0  600.0  700.0  725.0  750.0  775.0  800.0  825.0  850.0  875.0  900.0  1000.0  1100.0  1200.0
PressureList[atm]                     0.10  0.32  1.00  3.20  10.00  32.00  100.00
!
ModelEnergyLimit[kcal/mol]             800.00
EnergyStepOverTemperature              0.20
!
CalculationMethod                      direct
!
WellCutoff                             10
!
ChemicalEigenvalueMax                  0.20
!
ReductionMethod                        diagonalization
!
AtomDistanceMin[angstrom]              0.68793
!
RateOutput                             rate.out
!
WellExtension      0.0010
!
ExtensionCorrection      0.6
!
!
!===================================================
!  BEGIN MASTER EQUATION MODEL
!===================================================
!
Model
!
GroundEnergyShiftMax[kcal/mol]  10
!
UseSho